In [2]:
import os
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch

# 파일 경로 설정
ABBREVIATION_FILE = './data/abbreviations.csv'
ABBREVIATION_FILE
#SIMPLIFIED_FILE = './data/simplified.csv'
#RESULT_FILE = './data/result.txt'
#DATA_DIR = './data'


RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
name '_C' is not defined

In [ ]:
import os
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch

# 파일 경로 설정
ABBREVIATION_FILE = 'abbreviation.csv'
SIMPLIFIED_FILE = 'simplified.csv'
RESULT_FILE = 'result.txt'
DATA_DIR = './data'

# 디렉토리 생성 (없으면 생성)
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

# CSV 파일 읽기
abbreviation_df = pd.read_csv(ABBREVIATION_FILE)
simplified_df = pd.read_csv(SIMPLIFIED_FILE)

# 토크나이저와 모델 초기화 (예: BERT 모델 사용)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# 줄임말 변환 함수
def replace_abbreviations(text, abbreviation_df):
    for _, row in abbreviation_df.iterrows():
        abbreviation = row['Abbreviation']
        full_term = row['Full Term']
        
        # 한국어 줄임말 처리
        if re.match(r'[가-힣]', abbreviation):
            text = re.sub(rf'\b{abbreviation}\b', full_term, text)
        # 영어 줄임말 처리
        else:
            text = re.sub(rf'\b{abbreviation}\b', f"{abbreviation}({full_term})", text)
    
    return text

# 용어 설명 추가 함수
def add_definitions(text, simplified_df):
    definitions = []
    for _, row in simplified_df.iterrows():
        term = row['Term']
        definition = row['Definition']
        impact = row['Impact']
        
        # 텍스트에 해당 용어가 있는 경우 추가
        if term in text:
            definitions.append(f"{term} : {definition}\n* 영향 : {impact}")
    
    if definitions:
        text += "\n\n" + "\n\n".join(definitions)
    
    return text

# 결과 텍스트 전처리 함수
def preprocess_text(text, abbreviation_df, simplified_df):
    # 1. 줄임말 변환
    text = replace_abbreviations(text, abbreviation_df)
    
    # 2. 용어 설명 추가
    text = add_definitions(text, simplified_df)
    
    return text

# 파일 전처리 함수 (result.txt 파일을 읽어서 전처리)
def preprocess_file(file_path, abbreviation_df, simplified_df, output_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    
    # 텍스트 전처리
    preprocessed_text = preprocess_text(text, abbreviation_df, simplified_df)
    
    # 전처리된 텍스트 저장
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(preprocessed_text)
    
    return preprocessed_text

# 토크나이즈 함수
def tokenize_text(text, tokenizer):
    # BERT 토크나이저로 토큰화
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    return inputs

# 모델 학습용 데이터셋 준비 함수
def prepare_dataset(text, tokenizer):
    tokenized_data = tokenize_text(text, tokenizer)
    dataset = Dataset.from_dict(tokenized_data)
    return dataset

# 학습 함수 (여기서는 학습 예시를 보여주지만, 실제로는 추가 데이터가 필요)
def train_model(train_dataset, model, tokenizer):
    training_args = TrainingArguments(
        output_dir='./results',          # 출력 디렉터리
        num_train_epochs=3,              # 학습 에폭 수
        per_device_train_batch_size=16,  # 배치 사이즈
        logging_dir='./logs',            # 로그 디렉터리
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset
    )
    
    trainer.train()

# 모델 테스트 함수
def test_model(test_text, model, tokenizer):
    # 전처리 및 토크나이즈
    preprocessed_test_text = preprocess_text(test_text, abbreviation_df, simplified_df)
    inputs = tokenize_text(preprocessed_test_text, tokenizer)
    
    # 모델 예측
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
    
    return outputs

# 예제: result.txt 파일 전처리 및 모델 학습, 테스트
def main():
    # 1. result.txt 파일을 전처리하고 ./data 폴더에 저장
    output_file_path = os.path.join(DATA_DIR, 'preprocessed_result.txt')
    preprocessed_text = preprocess_file(RESULT_FILE, abbreviation_df, simplified_df, output_file_path)
    
    # 2. 전처리된 텍스트를 토큰화하여 데이터셋 준비
    train_dataset = prepare_dataset(preprocessed_text, tokenizer)
    
    # 3. 모델 학습 (예시)
    train_model(train_dataset, model, tokenizer)
    
    # 4. 모델 테스트 예시
    test_text = "한은은 GDP가 증가하였다라고 발표했다."
    outputs = test_model(test_text, model, tokenizer)
    
    print("Test Outputs:", outputs)

if __name__ == "__main__":
    main()
